# Data Mining Versuch Fahrzeugdaten

* Autor: Prof. Dr. Johannes Maucher
* Datum: 28.06.2021


## Lernziele:
In diesem Notebook sollen Kenntnisse in folgenden Themen vermittelt werden:

* Datenimport aus .csv
* Explorative Datenanalysen (EDA)
* Datenvisualisierung mit Matplotlib und plotly
* Überwachtes Lernen eines Klassifikationsmodells
* Überwachtes Lernen eines Regressionsmodells
* Evaluation von Klassifikationsmodellen
* Evaluation von Regressionsmodellen
* Kreuzvalidierung
* Hyperparameteroptimierung

## Datenzugriff


Die Daten sollen aus der Datei `Fahrzeuginformationen.csv` eingelesen werden. Das benötigte File liegt im Verzeichnis `../Data`.

1. Laden Sie die .csv-Datei in einen Pandas Dataframe. 

2. Zeigen Sie für den angelegten Dataframe 
    * die ersten 10 Zeilen
    * die Größe (Anzahl Zeilen und Anzahl Spalten)
    * die Anzahl der NaNs pro Spalte
    an. 
3. Zeigen Sie mit der Pandas-Dataframe Methode `info()`, den Stellen Sie sicher, dass die Spalte `CO2-Emissionen` ein numerischer Typ (float) ist. 

In [1]:
from matplotlib import pyplot as plt
import plotly.express as px
import seaborn as sns
import pandas as pd
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)
import json

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, GridSearchCV, GroupKFold
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, MultiLabelBinarizer
from sklearn.metrics import plot_roc_curve, plot_confusion_matrix, classification_report
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import tree

## Data Exploration
1. Zeigen Sie für alle Spalten die Anzahl der unterschiedlichen Werte in dieser Spalte an.
2. Benutzen Sie die Pandas Dataframe Methode `describe()` um sämtliche deskriptiven Statistiken anzuzeigen.
3. Legen Sie eine Liste `numeric_features` an, welche nur die Spaltennamen der numerischen Spalten enthält.
4. Schreiben Sie die Namen aller nicht-numerischen Spalten in eine Liste `categoric_features`.
5. Visualisieren Sie für die Spalten `HST_Benennung`, `Neupreis Brutto`, `CO2-Emissionen` und `Produktgruppe` die Verteilung der Werte in einem Barplot bzw. Histogramm.

## Machine Learning 1: Produktgruppenbestimmung

In diesem Abschnitt soll ein Klassifikator trainiert werden, welcher anhand von Eingabemerkmalen, wie *Breite*, *Höhe*, *Gewicht* usw. das zugehörige Fahrzeugsegment (`Produktgruppe`) vorhersagt.

In diesem Teilversuch sollen als Eingabemerkmale die zuvor in `numeric_features` definierten Spalten und die nicht-numerischen Spalten `Antrieb`, `Kraftstoffart`, `KSTA Motor` verwendet werden. Die Zielvariable (Ausgabe) stellt die Spalte `Produktgruppe` dar.


Auswahl der relevanten Merkmale

### Data Encoding

1. Kategoriale Merkmale ohne Ordnungsrelation (=nominale Merkmale) müssen One-Hot-Encodiert werden. Führen Sie für die drei categorialen Merkmale ein One-Hot-Encoding mit dem [scikit-learn LabelBinarizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html) durch.
2. Fügen Sie die one-hot-encodierten Spalten mit den numerischen Spalten zusammen. Weisen Sie die entsprechende Eingabedatenmatrix einem 2-dimensionalen numpy-array `X` zu. 
3. Führen Sie auf die Zielvariable `Produktgruppe` ein Label-Encoding mit [scikit-learn LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) aus. Weisen Sie diese Daten dem 1-dimensionalen numpy-array `y` zu.

### Erzeuge Training- and Testpartition
Benutzen Sie die [scikit-learn Methode train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) um `X` und `y` in einer Trainings- und Testpartition aufzuteilen. 30% der Daten soll für das Testen, 70% für das Training benutzt werden.

In [24]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import plot_confusion_matrix, classification_report

### Decision Tree Training, Test and Evaluation
1. Trainieren Sie einen [Entscheidungsbaum](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) mit den Trainingsdaten.
2. Wenden Sie den gelernten Entscheidungsbaum auf die Testdaten.
3. Evaluieren Sie die Qualität des Entscheidungsbaumes indem Sie 
     - einen [classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) erzeugen. 
     - die [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html) plotten.
 
 Interpretieren Sie das Ergebnis.
 
4. Führen Sie eine [10-fache Kreuzvalidierung](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) des Entscheidungsbaumes mit den Daten `X` und `y` aus. Interpretieren Sie das Ergebnis.
5. Bestimmen Sie die *Wichtigkeit* der Eingabemerkmale für die Klassifikationsaufgabe, indem Sie auf den in 1.) gelernten DecisionTree das Attribut `feature_importance_` abfragen. Stellen Sie die Werte in einem Barplot dar.

### Random Forest Training, Test and Evaluation
Wiederholen Sie die Teilaufgaben 1. bis 5. des Entscheidungsbaums für einen [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). Vergelichen Sie die Performance der beiden Verfahren.

## Machine Learning 2: Schätzung der CO2-Emission
In diesem Teilversuch soll aus den Eingabemerkmalen 

`"CCM","HST PS", "Anzahl der Türen", "Leergewicht", "Zuladung", "Länge", "Breite", "Höhe"`

die Zielvariable 

`CO2-Emissionen`

geschätzt werden. Hierzu soll ein möglichst gutes Regressionsmodell trainiert werden.

### Visuelle Korrelationsanalyse
1. Stellen Sie für jedes der 8 Eingabemerkmale die Korrelation mit der Zielvariablen visuell in einem Scatterplot dar, in dem das jeweilige Eingabemerkmal auf der x-Achse und die Zielvariable auf der y-Achse aufgetragen wird.
2. Diskutieren Sie die Korrelationen. Welche Merkmale korrelieren am stärksten mit der Zielvariable? Erscheint Ihnen das plausibel?

In [37]:
co2_inputcols=["CCM","HST PS", "Anzahl der Türen", "Leergewicht", "Zuladung", "Länge", "Breite", "Höhe"] 

### Data Encoding
1. Weisen Sie die Matrix der Eingabedaten dem 2-dimensionalen Array `X` und die Zielvariable dem 1-dimensionalen Array `y` zu.
2. Führen Sie auf `X` und `y` eine Partitionierung in Trainings- und Testdaten durch, wieder im Verhältnis 70/30.
3. Skalieren Sie die Eingabevariablen und die Zielvariable mit dem [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html). Die Skalierung muss sowohl auf Trainings- als auch auf Testdaten ausgeführt werden. Warum darf die Skalierung erst nach dem Split in die beiden Partitionen ausgeführt werden? Worauf ist zu achten? 

### Training, Test und Evaluation verschiedener Regressionsmodelle

Führen Sie die folgenden Teilaufgaben sowohl für ein [Single Layer Perceptron](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html) als auch für ein [Multi Layer Perceptron](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) mit 20 Neuronen in der Hidden-Schicht durch. Vergleichen Sie am Ende die Performance der beiden Verfahren.
1. Trainieren Sie den Algorithmus mit den Trainingsdaten.
2. Wenden Sie das gelernte Modell auf die Testdaten an.
3. Evaluieren Sie die Qualität der Modelle, indem Sie auf die vorhergesagten Ausgaben und die wahren Ausgaben die unten gegebene Funktion aufrufen.
4. Beschreiben Sie kurz die in der Funktion verwendeten Metriken

In [49]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

In [50]:
def determineRegressionMetrics(y_test,y_pred,title=""):
    mse = mean_squared_error(y_test, y_pred)
    mad = mean_absolute_error(y_test, y_pred)
    rmsle=np.sqrt(mean_squared_error(np.log(y_test+1),np.log(y_pred+1)))# +1 for avoiding log(0) 
    r2=r2_score(y_test, y_pred)
    med=median_absolute_error(y_test, y_pred)
    print(title)
    print("Mean absolute error =", round(mad, 2))
    print("Mean squared error =", round(mse, 2))
    print("Median absolute error =", round(med, 2))
    print("R2 score =", round(r2, 2))
    print("Root Mean Squared Logarithmic Error =",rmsle)

In [51]:
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor

### Hyperparameteroptimierung

Für ein [Multi Layer Perceptron](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) soll eine Hyperparameteroptimierung durchgeführt werden. Ziel ist es innerhalb der unten vorgegebenen Wertebereiche für die Hyperparameter `hidden_layer_sizes`, `activation` und `learning_rate` die beste Konfiguration zu finden. Hierzu kann entweder [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) oder [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) eingesetzt werden. GridSearchCV testet einfach alle Konfigurationen durch, benötigt daher aber viel Zeit. RandomizedSearchCV geht heuristisch und damit schneller durch den Suchraum. Wenden Sie eines dieser beiden Verfahren an, um für das unten gegebene Parameter-Grid die optimale Konfiguration zu finden. Welches ist die optimale Konfiguration und zu welchem `neg_mean_absolute_error`  führt diese?

In [60]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [61]:
param_grid = [{'hidden_layer_sizes': [(10,),(20,),(30,),(40,),(50,),(100,),(10,10)], 
               'activation': ["logistic", "tanh", "relu"], 
               'learning_rate': ["constant", "invscaling", "adaptive"]}]
param_grid

[{'hidden_layer_sizes': [(10,), (20,), (30,), (40,), (50,), (100,), (10, 10)],
  'activation': ['logistic', 'tanh', 'relu'],
  'learning_rate': ['constant', 'invscaling', 'adaptive']}]